In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report


In [ ]:
# Step 0: Loading Data
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
dow_jones_index = fetch_ucirepo(id=312) 
  
# data (as pandas dataframes) 
X = dow_jones_index.data.features 
y = dow_jones_index.data.targets 
  
# metadata 
print(dow_jones_index.metadata) 
  
# variable information 
print(dow_jones_index.variables) 
